In [62]:
# ============================================================
# Imports
# ============================================================

!pip install gensim  # Install gensim for Word2Vec
from gensim.models import Word2Vec  # Word2Vec embeddings
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split  # Split dataset into train/test

In [63]:
# ============================================================
# Load and Prepare Dataset
# ============================================================

# Load Steam reviews CSV
df = pd.read_csv("elden_ring_steam_reviews.csv")

# Keep only columns relevant to sentiment analysis
df = df[["review", "voted_up", "language"]]

# Filter to English reviews only
df = df[df["language"] == "english"]

# Convert voted_up (True/False) into binary labels (1 = positive, 0 = negative)
df["sentiment"] = df["voted_up"].astype(int)

# Remove missing or empty reviews
df = df.dropna(subset=["review"])
df = df[df["review"].str.strip() != ""]

# Extract review text (X) and sentiment labels (y)
X = df["review"].to_numpy()
y = df["sentiment"].to_numpy()

print(f"Total samples: {len(X)}")  # Check dataset size

Total samples: 9763


In [64]:
# ============================================================
# Text Preprocessing
# ============================================================

def tokenize(text):
    """
    Convert text into lowercase word tokens.
    Removes punctuation and non-letter characters to reduce noise.
    """
    text = text.lower()
    text = re.sub(r"[^a-z\s]", "", text)
    return text.split()

In [65]:
# ============================================================
# Train / Test Split
# ============================================================

# Split reviews into training (80%) and testing (20%) sets
# random_state ensures reproducible splits
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Train size: {len(X_train)}")
print(f"Test size:  {len(X_test)}")



Train size: 7810
Test size:  1953


In [66]:
# Tokenize training reviews for Word2Vec
corpus = [tokenize(text) for text in X_train]
print(corpus)  # Optional: inspect tokenized sentences

[['note', 'i', 'played', 'this', 'game', 'at', 'launch', 'its', 'okay', 'this', 'review', 'will', 'compare', 'elden', 'ring', 'to', 'dark', 'souls', 'iii', 'a', 'lot', 'for', 'obvious', 'reasons', 'the', 'open', 'world', 'formula', 'hurts', 'this', 'game', 'a', 'lot', 'a', 'sense', 'of', 'bewilderment', 'is', 'common', 'when', 'you', 'move', 'around', 'with', 'barely', 'any', 'direction', 'at', 'all', 'you', 'get', 'indications', 'to', 'sites', 'of', 'grace', 'but', 'thats', 'it', 'when', 'i', 'tried', 'going', 'someplace', 'interesting', 'on', 'my', 'own', 'i', 'got', 'stomped', 'instantly', 'if', 'i', 'didnt', 'explore', 'the', 'crevices', 'of', 'the', 'vast', 'empty', 'land', 'i', 'was', 'under', 'leveled', 'if', 'i', 'explored', 'everything', 'then', 'i', 'saw', 'the', 'same', 'ruins', 'asset', 'and', 'enemies', 'scattered', 'everywhere', 'it', 'felt', 'like', 'a', 'grind', 'and', 'exploration', 'for', 'the', 'sake', 'of', 'exploration', 'when', 'i', 'moved', 'on', 'to', 'the', 'co

In [67]:
# ============================================================
# Train Word2Vec Model
# ============================================================

# Word2Vec learns vector representations for words
# based on their surrounding context in the training corpus
model = Word2Vec(
    corpus,
    vector_size=50,   # Dimensionality of word embeddings. The number of dimensions in each word’s embedding vector
    #Fewer parameters → faster training, less memory, but less capacity to capture subtle semantic differences.
    window=4,         # Context window size
    min_count=5,      # Ignore rare words
    sg=1              # Use Skip-Gram (better for semantics) Predict surrounding words given a target word
)

In [68]:
# ============================================================
# Sentence Vector Construction
# ============================================================

def sentence_vector(tokens):
    """
    Convert a list of tokens into a single vector for the sentence.
    - Averages embeddings of known words.
    - Returns zero vector if no known words are found.
    """
    vecs = [model.wv[w] for w in tokens if w in model.wv]
    if not vecs:
        return np.zeros(model.vector_size)  # Handle rare/unseen words
    return np.mean(vecs, axis=0)

In [69]:
# Convert all training sentences into vectors
X_train_vecs = np.array([
    sentence_vector(tokenize(text)) for text in X_train
])

y_train = np.array(y_train)

In [70]:
# ============================================================
# Build Sentiment Prototypes
# ============================================================

# Positive prototype: average vector of all positive reviews
pos_vec = np.mean(X_train_vecs[y_train == 1], axis=0)

# Negative prototype: average vector of all negative reviews
neg_vec = np.mean(X_train_vecs[y_train == 0], axis=0)

In [71]:
# ============================================================
# Similarity and Prediction
# ============================================================

def cosine(a, b):
    """
    Compute cosine similarity between two vectors in a numerically safe way.

    WHY THIS FUNCTION EXISTS:
    -------------------------
    Cosine similarity is defined as:

        cos(theta) = (a · b) / (||a|| * ||b||)

    However, this formula breaks when either vector has zero magnitude.
    In this project, zero vectors occur when:
      - A sentence contains no words seen by Word2Vec
      - All tokens were removed during preprocessing
      - The averaged prototype vector collapses toward zero

    In those cases, the denominator becomes zero and the result is NaN.
    NaN values silently poison comparisons and degrade accuracy.
    """

    # Compute the magnitude (Euclidean norm) of vector a
    norm_a = np.linalg.norm(a)

    # Compute the magnitude (Euclidean norm) of vector b
    norm_b = np.linalg.norm(b)

    # Guard clause: if either vector has zero length,
    # cosine similarity is undefined.
    #
    # Instead of allowing a division-by-zero (0 / 0),
    # we return 0.0, meaning "no similarity".
    #
    # This choice is conservative and prevents NaN values
    # from propagating into the classifier logic.
    if norm_a == 0 or norm_b == 0:
        return 0.0

    # If both vectors are valid, compute the cosine similarity normally
    return np.dot(a, b) / (norm_a * norm_b)

def predict(sentence):
    """
    Predict sentiment for a single sentence.

    Steps:
    1. Tokenize and vectorize the input sentence
    2. Compare it to positive and negative prototypes
    3. Assign the sentiment with higher cosine similarity
    """

    # Convert raw text into word tokens
    tokens = tokenize(sentence)

    # Convert tokens into a single averaged sentence vector
    v = sentence_vector(tokens)

    # Measure similarity to the positive sentiment prototype
    sim_pos = cosine(v, pos_vec)

    # Measure similarity to the negative sentiment prototype
    sim_neg = cosine(v, neg_vec)

    # Whichever prototype is closer determines the prediction
    if sim_pos > sim_neg:
        return "positive"
    else:
        return "negative"

In [72]:
# ============================================================
# Evaluation on Test Set
# ============================================================

# Counter to track correct predictions
correct = 0

# Limit how many predictions we print (for readability)
display = 10

# Iterate over each test review and its true label
for text, label in zip(X_test, y_test):

    # Predict sentiment using prototype comparison
    guess = predict(text)

    # Convert numeric label into human-readable form
    answer = "positive" if label == 1 else "negative"

    # Print a few example predictions for inspection
    if display > 0:
        print(
            text[:80],          # Truncate long reviews for clarity
            "→", guess,
            "; Correct:", answer
        )
        display -= 1

    # Update accuracy counter
    if guess == answer:
        correct += 1


# Final accuracy is the fraction of correct predictions
accuracy = correct / len(X_test)

print("Accuracy:", accuracy)

AAAAAAAH-RISE NOW, YE TARNISHED → negative ; Correct: positive
I get to kill god → negative ; Correct: positive
fockin gud yeah → positive ; Correct: positive
It's a very good game and overall a great experience. It will also keep you ente → positive ; Correct: positive
I am a 45 yo father, probably one of the oldest people playing this game. I am a → negative ; Correct: positive
Well deserved Game of the Year 2022! I had 120 Hours of pure excitement from jus → negative ; Correct: positive
Its peak. → positive ; Correct: positive
Absolutely love it!
Took a while to get the 100% though :D → negative ; Correct: positive
cool → positive ; Correct: positive
Item and mob randomiser mods are for confirmed gods. only.
no flask shards and n → negative ; Correct: positive
Accuracy: 0.4910394265232975


In [73]:
# ============================================================
# Manual Tests
# ============================================================

# Hand-written sentences to probe model behavior
# These help validate qualitative performance

tests = [
    "the game is hard but rewarding",
    "horribly optimized and constantly freezes",
    "good environment but horrible gameplay",
    "this is the best thing i have ever played"
]

print("\nManual tests:")
for t in tests:
    print(t, "→", predict(t))


Manual tests:
the game is hard but rewarding → positive
horribly optimized and constantly freezes → negative
good environment but horrible gameplay → positive
this is the best thing i have ever played → positive
